In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/videos'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/videos/L12_V017.mp4
/kaggle/input/videos/L11_V008.mp4
/kaggle/input/videos/L11_V012.mp4
/kaggle/input/videos/L11_V029.mp4
/kaggle/input/videos/L12_V002.mp4
/kaggle/input/videos/L11_V020.mp4
/kaggle/input/videos/L12_V007.mp4
/kaggle/input/videos/L12_V025.mp4
/kaggle/input/videos/L11_V028.mp4
/kaggle/input/videos/L12_V008.mp4
/kaggle/input/videos/L12_V018.mp4
/kaggle/input/videos/L12_V015.mp4
/kaggle/input/videos/L12_V024.mp4
/kaggle/input/videos/L11_V017.mp4
/kaggle/input/videos/L12_V029.mp4
/kaggle/input/videos/L12_V028.mp4
/kaggle/input/videos/L11_V010.mp4
/kaggle/input/videos/L12_V020.mp4
/kaggle/input/videos/L12_V014.mp4
/kaggle/input/videos/L11_V011.mp4
/kaggle/input/videos/L11_V021.mp4
/kaggle/input/videos/L11_V001.mp4
/kaggle/input/videos/L12_V021.mp4
/kaggle/input/videos/L11_V014.mp4
/kaggle/input/videos/L11_V003.mp4
/kaggle/input/videos/L12_V022.mp4
/kaggle/input/videos/L11_V015.mp4
/kaggle/input/videos/L12_V030.mp4
/kaggle/input/videos/L12_V005.mp4
/kaggle/input/

In [2]:
!pip install opencv-python-headless numpy
!git clone https://github.com/soCzech/TransNetV2
%cd TransNetV2/inference
# !pip install google

Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 362 (delta 70), reused 70 (delta 70), pack-reused 278 (from 1)
Receiving objects: 100% (362/362), 95.27 KiB | 1.29 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Filtering content: 100% (3/3), 34.77 MiB | 21.77 MiB/s, done.
/kaggle/working/TransNetV2/inference


In [ ]:
import os
import cv2
import json
import zipfile
from transnetv2 import TransNetV2
from IPython.display import FileLink, HTML


In [ ]:
video_paths = []
for dirname, _, filenames in os.walk('/kaggle/input/videos'):
    for filename in filenames:
        video_path = os.path.join(dirname, filename)
        if os.path.exists(video_path):
            video_paths.append(video_path)
video_paths.sort()
print(video_paths)

In [ ]:
output_folder = '/kaggle/working/output'
    
if not os.path.exists(output_folder):
  os.makedirs(output_folder)

model = TransNetV2()

In [ ]:
def getSceneFrames(video_path: str):
    folder_name = video_path.split('/')[-1].replace('.mp4', '')
    
    result_folder = f'{output_folder}/{folder_name}'
    
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    
    keyframes_folder = f'{result_folder}/keyframes'
    
    if not os.path.exists(keyframes_folder):
        os.makedirs(keyframes_folder)

    keyframes_mapping_json = f'{result_folder}/{folder_name}.json'

    
    # Ensure the directory for JSON file exists, not the file itself
    child_mapping_dir = os.path.dirname(keyframes_mapping_json)
    if not os.path.exists(child_mapping_dir):
        os.makedirs(child_mapping_dir)
    
    # Make sure that no directory exists with the same name as the JSON file
    if os.path.isdir(keyframes_mapping_json):
        print(f"Error: {keyframes_mapping_json} is a directory, not a file.")
        return
       
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_resized = cv2.resize(frame_rgb, (48, 27))
            frames.append(frame_resized)
        else:
            break

    cap.release()

    frames = np.array(frames)

    predictions = model.predict_frames(frames)
    
    prediction, prediction_changes = predictions

    scene_changes = np.where(prediction_changes > 0.85)[0]

    scene_changes_updated = []

    for i in range(1, len(scene_changes)):
        if scene_changes[i] - scene_changes[i - 1] > 1:
            scene_changes_updated.append(scene_changes[i - 1])

    if scene_changes[-1] - scene_changes[-2] > 1:
        scene_changes_updated.append(scene_changes[-1])
    else:
        scene_changes_updated.pop()
        scene_changes_updated.append(scene_changes[-1])
    
    frames_mapping_dict = {}
    current_frames = []
    frame_index = 0
    index = 0
    
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index in scene_changes or frame_index == len(frames) - 1:
            len_frames = len(current_frames)
            if len_frames > 2:
                start_frame = current_frames[0]
                middle_frame = current_frames[len_frames // 2]
                end_frame = current_frames[len_frames - 1]

                cv2.imwrite(os.path.join(keyframes_folder, f"{index}.jpg"), start_frame)
                cv2.imwrite(os.path.join(keyframes_folder, f"{index + 1}.jpg"), middle_frame)
                cv2.imwrite(os.path.join(keyframes_folder, f"{index + 2}.jpg"), end_frame)

                frames_mapping_dict[index] = frame_index - len_frames
                frames_mapping_dict[index + 1] = frame_index - len_frames + len_frames // 2
                frames_mapping_dict[index + 2] = frame_index - 1

                current_frames = []
                index += 3

        else:
            current_frames.append(frame)
            
        frame_index += 1

    cap.release()
        
    # Write the mapping to the JSON file
    with open(keyframes_mapping_json, 'w') as json_file:
        json.dump(frames_mapping_dict, json_file, indent=4)
    
    print(f"Lưu thành công {index // 3} đoạn video và 1 file keyframe-mapping vào thư mục {result_folder}")


In [ ]:
def compress_to_zip(video_path, output_folder):
    
    folder_name = video_path.split('/')[-1].replace('.mp4', '')
    
    input_folder = f'{output_folder}/{folder_name}'

    # Đặt tên cho file zip đầu ra
    output_zip = f'/kaggle/working/{folder_name}.zip'
    
    download_path = f"https://kkb-production.jupyter-proxy.kaggle.net/k/194493829/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..sKAZVNzwRVzG1ZFJGD_BbA.bFaQ9PSjVhClJHFXxNbGZHdnac7S5HYRS2bUq-FTjasns-6-j0lRyCsC8aGwIqsWbAuLamLiyBZr3sIOLRJ90MEj11lh1YHAbp6G9l9-3tK_NtyRY65jUkGmEC-HYkU-NBmcxiEVOmxelzvk9hHvkS9EGPjr2l6t7kegs5oTlMruRZOZq7LsrYQdifLfEoDlRD8-L7WKd_jDwG5Kt3qv9A.qsBo-R09cdVqCyJfodN8ZQ/proxy/files/{folder_name}.zip"
    
    # Tạo file zip
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(input_folder):
            for file in files:
                file_path = os.path.join(root, file)
                # Thêm file vào zip, giữ nguyên cấu trúc thư mục
                zipf.write(file_path, os.path.relpath(file_path, input_folder))

    print(f"Folder zipped to {output_zip}")
    
    # Hiển thị link tải về
    display(HTML(f'<a href="{download_path}" target="_blank">Click here to download {output_zip} ZIP file</a>'))


In [ ]:
def release_folder(video_path:str):
    folder_name = video_path.split('/')[-1].replace('.mp4', '')
    
    result_folder = f'{output_folder}/{folder_name}'
    
    folder_path = f'{result_folder}/keyframes'
    
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f'Deleted file: {file_path}')
    else:
        print('The specified folder does not exist')

In [ ]:
for video_path in video_paths:
    getSceneFrames(video_path)
    compress_to_zip(video_path, output_folder)

# getSceneFrames(video_paths[0])
# compress_to_zip(output_folder, output_zip)

# release_folder(video_paths[0])
# release_folder(video_paths[1])